In [1]:
from rdflib import Graph
from rdflib.namespace import RDF, RDFS, DCTERMS, FOAF, SKOS, XSD

In [87]:
ontologyFN = "./elm.ttl"

In [3]:
g = Graph()
g.parse(ontologyFN)
print(len(g))

1535


In [88]:
for elmClass, p, o in g.triples((None, RDF.type, RDFS.Class)):
    for identifier in g.objects(elmClass, DCTERMS.identifier, None):
        class_id = identifier.split(":")[1]
    for name in g.objects(elmClass, RDFS.label, None):
        class_name = name
    print("%%======")
    print("%%",class_name)
    print("%%======")
    print(f"    class {class_id}[\"{class_name}\"]{{")
    remainders = []
    relations = []
    dates = []
    durations = []
    concepts = []
    strings = []
    integers = []
    positive_integers = []
    booleans = []
    objects = []
    remainders = []
    for prop, p, o in g.triples((None, RDFS.domain, elmClass)):
        remainders.append(prop)
        for prop_id in g.objects(prop, DCTERMS.identifier, None):
            prop_token = prop_id.split(":")[1]
            for prop_range in g.objects(prop, RDFS.range):
                if prop_range == SKOS.Concept:
                    concepts.append(prop_token)
                    remainders.remove(prop)
                elif prop_range == XSD.string: 
                    strings.append(prop_token)
                    remainders.remove(prop)
                elif prop_range == XSD.dateTime: 
                    dates.append(prop_token)
                    remainders.remove(prop)
                elif prop_range == XSD.duration: 
                    durations.append(prop_token)
                    remainders.remove(prop)
                elif prop_range == XSD.integer: 
                    integers.append(prop_token)
                    remainders.remove(prop)
                elif prop_range == XSD.positiveInteger: 
                    positive_integers.append(prop_token)
                    remainders.remove(prop)
                elif prop_range == XSD.boolean: 
                    booleans.append(prop_token)
                    remainders.remove(prop)
                else:
                    objects.append([prop, prop_token, prop_range])
                    remainders.remove(prop)
    for prop_token in strings:
        print(f"        +String   {prop_token}")
    for prop_token in dates:
        print(f"        +Date     {prop_token}")
    for prop_token in durations:
        print(f"        +Duration {prop_token}")
    for prop_token in integers:
        print(f"        +Integer  {prop_token}")
    for prop_token in positive_integers:
        print(f"        +PositiveInteger    {prop_token}")    
    for prop_token in booleans:
        print(f"        +Boolean {prop_token}")
    for prop_token in concepts:
        print(f"        +Concept  {prop_token}")
    print("    }")
    for prop_info in objects:
        (prop, prop_token, prop_range) = prop_info
        for identifier in g.objects(prop_range, DCTERMS.identifier, None):
            range_id = identifier.split(":")[1]
            
        print(f"    {class_id} --> {range_id} : {prop_token}")
    if len(remainders) > 0:
        print("Remaining properties: ", remainders)


%%======
%% Accreditation
%%======
    class Accreditation["Accreditation"]{
        +String   decision
        +Date     reviewDate
        +Concept  limitField
        +Concept  limitEQFLevel
        +Concept  limitJurisdiction
        +Concept  limitCredentialType
        +Concept  limitAbstractProgramme
    }
    Accreditation --> WebResource : report
    Accreditation --> Organisation : organisation
    Accreditation --> Qualification : limitQualification
    Accreditation --> Organisation : accreditingAgent
    Accreditation --> WebResource : landingPage
%%======
%% Address
%%======
    class Address["Address"]{
    }
%%======
%% Agent
%%======
    class Agent["Agent"]{
    }
    Agent --> ContactPoint : contactPoint
%%======
%% Amount
%%======
    class Amount["Amount"]{
        +Concept  unit
    }
%%======
%% Awarding Body
%%======
    class AwardingBody["Awarding Body"]{
    }
%%======
%% Awarding Opportunity
%%======
    class AwardingOpportunity["Awarding Opportunity"]{
   

## Process the textual notes about reused properties

In [135]:
with open(ontologyFN, "r") as f:
    lines = f.readlines()
    
for line in lines:
    if line[0] == "#":
        if (" properties " in line) and ("#----" in line) :
            class_id = line.split(" properties")[0].split(" Properties")[0].split("---- ")[1].replace(" ", "")
            print("\n======\n",class_id,"\n======")
        if line[0:6] == "## Re-":
            pass
        elif line[0:4] == "### ": #these are lists of re-used properties
            props = line[4:-1].split(", ")
            for prop in props:
                prop = prop.replace(";","")
                prop = prop.replace(",","")
                if prop == "adms:identifier":
                    print("        +String adms_identifier")
                elif prop == "dc:description":
                    print("        +String dc_description")
                elif prop == "dc:title":
                    print("        +String dc_title")
                elif prop == "dc:issued":
                    print("        +Date dc_issued")
                elif prop == "dc:modified":
                    print("        +Date dc_modified")
                elif prop == "dc:valid":
                    print("        +Date dc_valid")
                elif prop == "elm:endDate":
                    print("        +Date endDate")
                elif prop == "dc:type":
                    print("        +Concept dc_type")
                elif prop == "elm:status":
                    print("        +Concept status")
                elif prop == "elm:additionalNote":
                    print(f"   {class_id} --> Note : additionalNote")
                elif prop == "elm:supplementaryDocument":
                    print(f"   {class_id} --> WebResource : supplementaryDocument")
                elif prop == "foaf:homepage":
                    print(f"   {class_id} --> WebResource : foaf_homepage")
                elif prop == "skos:altLabel":
                    print("        +String skos_altLabel")
                elif prop == "skos:prefLabel":
                    print("        +String skos_prefLabel")
                elif prop == "elm:location":
                    print(f"   {class_id} --> dc_Location : location")
                elif prop == "elm:lastModificationDate":
                    print("        +Date lastModificationDate")
                elif prop == "elm:learningAchievementSpecification":
                    print(f"   {class_id} --> LearningAchievementSpecification : learningAchievementSpecification")
                elif prop[0:11] == "dc:temporal":
                    print(f"   {class_id} --> dc_PeriodOfTime : dc_temporal")
                elif prop == "elm:hasPart":
                    print(f"   {class_id} --> {class_id} : hasPart")
                elif prop == "elm:address":
                    print(f"   {class_id} --> Address : address")
                elif prop == "elm:awardedBy":
                    print(f"   {class_id} --> AwardingProcess : awardedBy")
                elif prop == "elm:address":
                    print(f"   {class_id} --> Address : address")
                elif prop == "elm:supplementaryDocument":
                    print(f"   {class_id} --> WebResource : supplementaryDocument")
                elif prop == "elm:specifiedBy":
                    print(f"   {class_id} --> {class_id}Specification : specifiedBy")
                elif prop == "elm:expiryDate":
                    print("        +Date expiryDate")
                elif prop == "skos:notation":
                    print("        +String skos_notation")
                elif prop == "dc:creator":
                    print(f"   {class_id} --> Agent : dc_creator")
                elif prop == "adms:schemeAgency":
                    print("        +String adms_schemeAgency")
                elif prop == "elm:schemeName":
                    print("        +String adms_schemeName")
                elif prop == "elm:schemeVersion":
                    print("        +String adms_schemeVersion")
                elif prop == "dc:spatial":
                    print("        +Concept dc_spatial")
                elif prop == "rov:legalName":
                    print("        +String rov_legalName")
                elif prop == "rov:registration":
                    print("        +String rov_registration")
                elif prop == "elm:accreditation":
                    print(f"   {class_id} --> Accreditation : accreditation")
                elif prop == "elm:contactPoint":
                    print(f"   {class_id} --> ContactPoint : contactPoint")
                elif prop == "org:hasSubOrganization":
                    print(f"   {class_id} --> Organization : org_hasSubOrganization")
                elif prop == "org:subOrganizationOf":
                    print(f"   {class_id} --> Organization : org_subOrganizationOf")
                elif prop == "org:hasMember":
                    print(f"   {class_id} --> Agent : org_hasMember")
                elif prop == "fullname->foaf:name":
                    print("        +String foaf_name")
                elif prop == "foaf:givenName":
                    print("        +String foaf_givenName")
                elif prop == "foaf:familyName":
                    print("        +String foaf_familyName")
                elif prop == "cpv:birthName":
                    print("        +String cpv_birthName")
                elif prop == "cpv:patronymicName":
                    print("        +String cpv_patronymicName")
                elif prop == "org:memberOf (with range rov:registeredOrganisation)":
                    print(f"   {class_id} --> Organization : org_memberOf")
                elif prop == "which has elm:startDate and elm:endDate":
                    pass
                elif prop == "elm:learningOpportunity":
                    print(f"   {class_id} --> LearningOpportunity : learningOpportunity")
                elif prop == "foaf:member":
                    print(f"   {class_id} --> Agent : foaf_hasMember")
                elif prop == "elm:awardingDate":
                    print("        +Date awardingDate")
                elif prop == "elm:awardingBody":
                    print(f"   {class_id} --> AwardingBody : awardingBody")
                elif prop == "elm:contentUrl":
                    print("        +URL contentUrl")
                elif prop == "dc:language":
                    print("        +Concept dc_language")
                elif prop == "cred:validFrom":
                    print("        +Date cred_validFrom")
                elif prop == "cred:validUntil":
                    print("        +Date cred_validUntil")
                elif prop == "cred:issuanceDate":
                    print("        +Date cred_issuanceDate")
                elif prop == "cred:expirationDate":
                    print("        +Date cred_expirationDate")
                elif prop == " cred:issuer":
                    print(f"   {class_id} --> Organisation : cred_issuer")
                elif prop == "cred:credentialSubject":
                    print(f"   {class_id} --> Person : cred_subject")
                elif prop == "cred:evidence":
                    print(f"   {class_id} --> WebResource : cred_evidence")
                elif prop == "cred:termsOfUse":
                    print(f"   {class_id} --> WebResource : cred_termsOfUse")
                elif prop == "cred:credentialSchema":
                    print(f"   {class_id} --> WebResource : cred_credentialSchema")
                elif prop == "cred:proof":
                    pass
                elif prop == "cred:credentialStatus":
                    print("        +Concept cred_credentialStatus")
                elif prop == "dct:type":
                    print("        +Concept dc_type")
                elif prop == "elm:category":
                    print("        +String category")
                elif prop == "elm:volumeOfLearning":
                    print("        +Duration volumeOfLearning")
                elif prop == "elm:mode":
                    print("        +Concept mode")
                elif prop == "elm:specialisationOf":
                    print(f"   {class_id} --> {class_id} : specialisationOf")
                elif prop == "elm:provenBy":
                    print(f"   {class_id} --> LearningAssessment : provenBy")
                elif prop == "elm:influencedBy":
                    print(f"   {class_id} --> LearningActivity : influencedBy")
                elif prop == "elm:entitlesTo":
                    print(f"   {class_id} --> LearningEntitlement : entitlesTo")
                elif prop == "elm:proves":
                    print(f"   {class_id} --> LearningAchievement : proves")
                elif prop == "elm:entitledBy":
                    print(f"   {class_id} --> LearningAssessment : proves")
                elif prop == "dc:subject":
                    print("        +Concept dc_subject")
                elif prop == "elm:learningOutcomeSummary":
                    print(f"   {class_id} --> Note : learningOutcomeSummary")
                elif prop == "elm:ISCEDFCode":
                    print("        +Concept ISCEDFCode")
                elif prop == "elm:creditPoint":
                    print(f"   {class_id} --> CreditPoint : creditPoint")
                elif prop == "elm:educationLevel":
                    print("        +Concept educationLevel")
                elif prop == "elm:educationSubject":
                    print("        +Concept educationSubject")
                elif prop == "elm:learningSetting":
                    print("        +Concept learningSetting")
                elif prop == "elm:maximumDuration":
                    print("        +Duration maximumDuration")
                elif prop == "elm:targetGroup":
                    print("        +Concept targetGroup")
                elif prop == "elm:entryRequirement":
                    print(f"   {class_id} --> LearningAchievementSpecification : entryRequirement")
                elif prop == "elm:learningOutcome":
                    print(f"   {class_id} --> LearningOutcome : learningOutcome")
                elif prop == "elm:awardingOpportunity":
                    print(f"   {class_id} --> AwardingOpportunity : awardingOpportunity")
                elif prop == "foaf:topic":
                    print("        +Concept foaf_topic")
                elif prop == "elm:influences":
                    print(f"   {class_id} --> LearningAchievement : influences")
                elif prop == "skos:inScheme":
                    print(f"   {class_id} --> skos_ConceptScheme : skos_inScheme")
        
                    
                else: 
                    print("***********unrecognised prop:..", prop,".")
        
        


 Accreditation 
        +String adms_identifier
        +Concept dc_type
        +String dc_title
        +String dc_description
        +Date dc_issued
   Accreditation --> Note : additionalNote
   Accreditation --> WebResource : supplementaryDocument
   Accreditation --> WebResource : foaf_homepage
        +Concept status
        +Date dc_modified
        +Date dc_valid

 Address 
        +String adms_identifier

 Agent 
        +String adms_identifier
        +String skos_altLabel
        +String skos_prefLabel
   Agent --> Note : additionalNote
   Agent --> dc_Location : location
        +Date lastModificationDate

 Group 
        +String adms_identifier
        +String skos_altLabel
        +String skos_prefLabel
        +Concept dc_type
   Group --> Note : additionalNote
   Group --> dc_Location : location
   Group --> ContactPoint : contactPoint
   Group --> Agent : foaf_hasMember

 AwardingOpportunity 
        +String adms_identifier
   AwardingOpportunity --> dc_Location : lo